In [1]:
import numpy as np
import pandas as pd
from itertools import product
from collections import Counter
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [2]:
def metric_calculation(pred, gt):    
    acc=accuracy_score(gt, pred)
    f1=f1_score(gt, pred, average='macro')
    confusion=confusion_matrix(gt, pred)
    fpr=confusion[0,1]/len(gt) ## predict to be 1; actual 0
    fnr=confusion[1,0]/len(gt) ## predict to be 0; actual 1
    return acc, f1, fpr, fnr

In [3]:
def post_processing(pred):
    new_pred=[]
    for i in pred:
        i=i.lower()
        if 'response' in i:
            try: new_pred.append(i.split('response')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        elif 'output' in i:
            try: new_pred.append(i.split('output')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        else:
            try: new_pred.append(i.split()[0].replace('</s>', ''))
            except:new_pred.append(2)
    new_pred = np.array([int(float(i)) if i in ['0', '0.0', '1', '1.0'] else 2 for i in new_pred])
    return new_pred

In [4]:
ds = load_dataset("beanham/spatial_union_dataset")
test=ds['test']
gt=np.array(test['label'])
configs = [
    'zero_shot_no_heur',    
    'zero_shot_with_heur_hint_angle',
    'zero_shot_with_heur_hint_area',
    'zero_shot_with_heur_hint_angle_area',
    'zero_shot_with_heur_value_angle',
    'zero_shot_with_heur_value_area',
    'zero_shot_with_heur_value_angle_area',
    'few_shot_no_heur',    
    'few_shot_with_heur_hint_angle',
    'few_shot_with_heur_hint_area',
    'few_shot_with_heur_hint_angle_area',
    'few_shot_with_heur_value_angle',
    'few_shot_with_heur_value_area',
    'few_shot_with_heur_value_angle_area'
]

In [5]:
results=[]
models=['llama3', '4o_mini', 'qwen']
for model in models:
    print(f'Model: {model}...')
    for config in configs:
        pred=np.load(f'base/{model}/{model}_{config}.npy')
        pred=post_processing(pred)        
        metrics=metric_calculation(pred, gt)
        results.append([config, model, round(metrics[0],3), metrics[1]])
results=pd.DataFrame(results, columns=['config', 'model', 'acc', 'f1'])

Model: llama3...
Model: 4o_mini...
Model: qwen...


In [6]:
results[results['model']=='qwen']

,config,model,acc,f1
28,zero_shot_no_heur,qwen,0.799,0.797143
29,zero_shot_with_heur_hint_angle,qwen,0.772,0.718371
30,zero_shot_with_heur_hint_area,qwen,0.852,0.835751
31,zero_shot_with_heur_hint_angle_area,qwen,0.752,0.750822
32,zero_shot_with_heur_value_angle,qwen,0.817,0.768975
33,zero_shot_with_heur_value_area,qwen,0.865,0.838340
34,zero_shot_with_heur_value_angle_area,qwen,0.927,0.917764
35,few_shot_no_heur,qwen,0.835,0.820162
36,few_shot_with_heur_hint_angle,qwen,0.822,0.808543
37,few_shot_with_heur_hint_area,qwen,0.827,0.792933
